In [ ]:
import numpy as np
import itertools
import statistics as stat
import math

# Input is a point, and returns all neighboring points without leaving [0,n]^d.
def neighborPoints(arr,n): #arr comes in as a tuple, n is size of boundary
    neighbors = []
    arr = list(arr)
    for i in range(len(arr)):
        if arr[i]+1 <=n:
            neighbors.append(arr[0:i] + [arr[i] +1] + arr[i+1:len(arr)])
        if arr[i]-1 >=0:
            neighbors.append(arr[0:i] + [arr[i] -1] + arr[i+1:len(arr)])
    return neighbors 

#Retuns a matrix M where M_ij is the total number of paths from the starting point to (i,j) in m steps.
def numberOfPaths(starting_point,d,n,m): #d = dimension, n = size, m = number of steps
    M_0 = np.zeros(tuple([n+1]*d)) 
    M_1 = np.zeros(tuple([n+1]*d))
    M_0[starting_point] = 1         #initialize M_0 as a matrix of zeros except a 1 at the starting point. 
    
    list_n = list(range(n+1))
    
    for i in range(m):
        for x in itertools.product(list_n, repeat=d): #loops through coordinates of the previous matrix
            total = 0
            for p in neighborPoints(x,n):
                total = total + M_0[tuple(p)]
            M_1[x] = total        
        M_0 = M_1
        M_1 = np.zeros(tuple([n+1]*d))
 
    return M_0

#Now we can use the funcitons to complete the challenge. 
# 1) d = 4, n = 10, m = 10. Find the total paths that start at the orgin. 

# gives the total number of valid paths from a given starting point. 
def totalNumberOfPaths(starting_point,d,n,m):
    total = numberOfPaths(starting_point,d,n,m)
    for i in range(d):
        total = sum(total)
        
    return total

print("The solution to problem #1 is :", totalNumberOfPaths((0,0,0,0),4,10,10))
        
# 2) d=4, n=10, m=10. Consider the count of valid walks at each starting position. Find ratio of SD to mean. 

#this function counts the number of times n appear in the coordinates of the point
def counter(n,point):
    c = 0
    for i in range(len(point)):
        if point[i] == n:
            c+=1
    return c

#returns list where each entry is the total number of paths associated to a starting point. 
def pathsByStartingPoint_Symmetry(d,n,m):
    data_list = []
    
    middle = math.ceil((n+1)/2)
    list_n = list(range(middle))
    
    i=1
    print("Steps until complete = ", middle**d)
    for x in itertools.product(list_n, repeat=d):
        for j in range(d+1):
            if counter(middle-1,x) ==j:
                data_list = data_list + [totalNumberOfPaths(x,d,n,m)]*(2**(d-j))
        print("\rStep: {}".format(i),end="")
        i+=1
        
    print("")
    return data_list

data_list = pathsByStartingPoint_Symmetry(4,10,10)
    
print("The solution to problem #2 is : ", stat.stdev(data_list)/stat.mean(data_list))

# 3) ratio of max and min, Simple adjustment to problem #2

print("The solution to problem #3 is: ", max(data_list)/min(data_list))

In [78]:
#While the solutions for the d=4, n = m = 10 case work, in a reasonable amount of time, the time scale
#becomes unmanagable for the d=8, n = m = 12 case. We will use some fancy combinatorics. It should be 
#noted that these more sophisticated methods would also work for d=4, n = m = 10 case. 

#So far this function only works for points along the diagonal of the d-cube. 
def totalNumberOfPaths_GeneratingFunction(starting_point, d, n, m):
    coef = [0]*(m+1)
    for i in range(0,m+1):
        x = totalNumberOfPaths(starting_point[0],1,n,i)
        coef[m-i] = x/math.factorial(i)

    poly = np.poly1d(coef)
    return round((poly**d)[m]*math.factorial(m))

maximum = totalNumberOfPaths_GeneratingFunction((6,6,6,6,6,6,6,6), 8, 12, 12)
minimum = totalNumberOfPaths_GeneratingFunction((0,0,0,0,0,0,0,0), 8, 12, 12)    

#so the answers to problems #4 and #6 are given:
    
print("The number of paths that start at a corner:", minimum)
print("The point (corner) with min number of paths has this many paths:", minimum)
print("The point (center) with max number of paths has this many paths:", maximum)

print("The ratio of Max number of paths to Min number of paths is:", maximum/minimum )




The number of paths that start at a corner: 2479092118272.0
The point (corner) with min number of paths has this many paths: 2479092118272.0
The point (center) with max number of paths has this many paths: 281467441879456.0
The ratio of Max number of paths to Min number of paths is: 113.53649983593472


In [ ]:
#Now we must do this over every starting point in {0,1,2,...,12}^8

d = 8
n = 12
m = 12

data_list =[]
middle = math.ceil((n+1)/2)
list_n = list(range(middle))

print("Steps until complete = ", middle**d)
num_of_steps=1

for starting_point in itertools.product(list_n, repeat=d):
    poly = [0]*d
    for j in range(0,d):
        coef = [0]*(m+1)
        for i in range(0,m+1):
            x = totalNumberOfPaths(starting_point[j],1,n,i)
            coef[m-i] = x/math.factorial(i)
        poly[j] = np.poly1d(coef)
    polynomial = np.poly1d([1])
    
    for k in range(d):
        polynomial = polynomial * poly[k]
        
    for l in range(d+1):
        if counter(middle-1,starting_point) == l:
            data_list = data_list + [round(polynomial[m]*math.factorial(m))]*(2**(d-l))
        
    print("\rStep: {}".format(num_of_steps),end="")
    num_of_steps+=1

In [ ]:
print("The solution to problem #5 is : ", stat.stdev(data_list)/stat.mean(data_list))